# SeaPACE tool
1. Select Study time + area / datasets (Currently PACE-PAX Aug 28 to Sep 30, 2024)
2. Confirm the match-up
3. Showing true color image and overlap of available datasets
4. Rrs plot
5. Selecting Subset of data

%pip install owslib
%pip install lxml

In [5]:
# Imports

import fsspec
import tarfile
import earthaccess

import holoviews as hv
from holoviews.streams import Tap
import matplotlib.pyplot as plt
import pandas as pd
import functions as fc
from step1 import get_OCI_PACE_truecolor

In [9]:
# Confirm your Earth Access login credentials

auth = earthaccess.login(strategy="interactive", persist=True)
session = auth.get_session()

____________________________________________________________________

## Step 1 - Download SeaBASS datafiles

##### After selecting your study region and time period, as well as your desired datatypes, request the data from SeaBASS

##### Use the .https link sent to download your data


In [20]:
import earthaccess
import tarfile
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# 1. Authenticate with Earthdata
auth    = earthaccess.login(strategy="interactive", persist=True)
session = auth.get_session()

# 2. Optional: retry adapter for transient HTTP errors
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))

# 3. Your NASA “getfile” URL
url = (
    "https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/"
    "requested_files_1.tar?h=ocdist404"
    "&p=/data1/00a0126131968662/requested_files"
)

# 4. Stream into tarfile, disable length enforcement, and extract member-by-member
with session.get(url, stream=True, timeout=(5, 120)) as r:
    r.raise_for_status()

    # Let urllib3 ignore any mismatch between header and actual bytes
    r.raw.decode_content = True
    if hasattr(r.raw, "enforce_content_length"):
        r.raw.enforce_content_length = False

    # Open the stream as a TarFile, then extract each member in turn
    tar = tarfile.open(fileobj=r.raw, mode="r|*")
    for member in tar:
        try:
            tar.extract(member, path="C:/Users/hunte/Downloads")
        except tarfile.ReadError as e:
            # Skip any files that really are truncated
            print(f"⚠️  Skipping '{member.name}' due to unexpected end of data")
    tar.close()

print("✅ Done (extracted all readable members)!")


✅ Done (extracted all readable members)!


______________________________________________________

## Step 2: 

In [6]:
#
#  Main configuration
#

# Specify the interest period, bounding box
tspan = ("2024-09-22", "2024-09-28")
bbox = (-125., 32., -116., 38.)

SeaBass_path = "/home/jovyan/shared-public/pace-hackweek/SeePACE/Hackweek_PACE-PAX_Rrs"  # SeaBass sb files (CCNY, GSFC, NRL, UCSB)

base_PACE_file = "PACE_OCI_L2_AOP"  # Rrs

In [ ]:
# Reading available SeaBass files and create dots on the worldview image

dates = fc.get_dates(tspan[0], tspan[1], 24)

In [ ]:
imglist = []
for date in dates:
    img = get_OCI_PACE_truecolor(date.strftime('%Y-%m-%d'), bbox)
    imglist.append(img)